### Notebook for constructiong POD-NODE NIROM approximation for a flow around a cylinder example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. POD is adopted to define a reduced basis space for the high-fidelity snaphosts. The evolution of the time dynamics in the POD-latent space is modeled using Neural ODEs (NODE).  

OpenFOAM is used as the high-fidelity model for simulating flow around a cylinder governed by incompressible 2D Navier Stokes.  

In [1]:
### Loading modules
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy
import os
import gc
import argparse
import platform
print("Python "+str(platform.python_version()))
import importlib
from importlib import reload as reload

import tensorflow as tf
print("Tensorflow "+ str(tf.__version__))
if tf.__version__ == '1.15.0':
    tf.compat.v1.enable_eager_execution()
elif tf.__version__.split('.')[0] == 2: 
    print("Setting Keras backend datatype")
    tf.keras.backend.set_floatx('float64')

from tfdiffeq import odeint,odeint_adjoint
from tfdiffeq.adjoint import odeint as adjoint_odeint
tf.keras.backend.set_floatx('float64')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
np.random.seed(0)

basedir   = os.getcwd()
srcdir = os.path.join(basedir,'../pynirom/')
workdir   = os.path.join(basedir,'../examples/')
datadir   = os.path.join(basedir,'../data/')
figdir    = os.path.join(basedir,'../figures')
nodedir   = os.path.join(basedir,'../data/')

modeldir = basedir
savedir = nodedir+'cylinder/current'

import pynirom
from pynirom.pod import pod_utils as pod
from pynirom.utils import data_utils as du
from pynirom.node import main as nd
from pynirom.node import plotting as pu
from pynirom.node import node as node
# os.chdir(workdir)



Python 3.8.0
Tensorflow 2.4.1
Num GPUs Available:  0


2021-07-17 04:06:30.940491: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [2]:
device = 'cpu:0' # select gpu:# or cpu:#
purpose= 'train' #'train' to train a new model and 'eval' to load a pre-trained model for evaluation (make sure you have the correct set of hyperparameters)
pre_trained_dir = nodedir+'/model_weights_cyl/' #If 'eval' specify path for pretrained model
stacking = True #stack or not
stack_order = 'v_x,v_y,p' #If stacking = True, specify the stacking order of the latent space vector
scale_time = False #Scale time or not (Normalize)
scale_states = True #Scale states or not (MinMax -1,1)
augmented,aug_dims = (False,5) #Augmented or not and size of augmentation
N_layers = int(1.0) #Only four layers supported as of now.
N_neurons = int(256) #Number of neurons per layer
act_f = 'tanh'  #Activation Function ('linear', 'tanh', 'sigmoid',...), default='linear'
learning_rate_decay = True #Use decaying learning rate or not
initial_learning_rate = float(0.001) #If 'learning_rate_decay = False' then this will be the fixed learning rate
decay_steps = int(5001) #Number of steps for learning rate decay
decay_rate = float(0.5) #Rate of learning rate decay
staircase_opt = True  #True for staircase decay and False for exponential
optimizer = 'RMSprop' #Adam and RMSprop optimizers are supported as of now
use_adjoint = False       #Use adjoint method or not
solver = 'rk4'     #Specify ODE solver. See tfdiffeq README for available options 
use_minibatch, batch_size = (False,256) #Use minibatch or not and batch size
epochs = int(10)     #Number of epochs of training


print("\n***** Runtime parameters: ******\n")
print(f'Mode = {purpose}, Scaling = {scale_states}, Augmenting = {augmented}, Adjoint = {use_adjoint}')
print(f'Solver = {solver}, Optimizer = {optimizer}, Stacking order = {stack_order}, Epochs = {epochs}')
print(f'# Layers = {N_layers}, # Neurons per layer = {N_neurons}, Activation fn = {act_f}')
print(f'Init LR = {initial_learning_rate}, # LR decay steps = {decay_steps}, LR decay rate = {decay_rate}')
print('**********************************\n')


***** Runtime parameters: ******

Mode = train, Scaling = True, Augmenting = False, Adjoint = False
Solver = rk4, Optimizer = RMSprop, Stacking order = v_x,v_y,p, Epochs = 10
# Layers = 1, # Neurons per layer = 256, Activation fn = tanh
Init LR = 0.001, # LR decay steps = 5001, LR decay rate = 0.5
**********************************



In [3]:
### ------ Import Snapshot data -------------------
data = np.load(datadir + 'cylinder_Re100.0_Nn14605_Nt3001.npz')
mesh = np.load(datadir + 'OF_cylinder_mesh_Nn14605_Ne28624.npz')

print('HFM data has {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    data['time'].shape[0],data['p'].shape[0],
                    data['time'][0], data['time'][-1]))


## ------- Prepare training snapshots ----------------
print('\n-------Prepare training and testing data---------')
soln_names = ['p', 'v_x', 'v_y']
nodes = mesh['nodes'];  node_ind = mesh['node_ind']
triangles = mesh['elems']; elem_ind = mesh['elem_ind']
snap_start = 1250
T_end = 5.0        ### 5 seconds
snap_incr = 4

snap_train, times_train = du.prepare_data(data, soln_names, start_skip=1250, T_end=5.0, incr=snap_incr)
print('Using {0} training snapshots for time interval [{1},{2}]'.format(times_train.shape[0],
                                        times_train[0], times_train[-1]))

## ------- Prepare testing snapshots ----------------
pred_incr = snap_incr -3
snap_pred_true, times_predict = du.prepare_data(data, soln_names, start_skip=snap_start, incr=pred_incr)
print('Using {0} testing snapshots for time interval [{1},{2}]'.format(times_predict.shape[0],
                                        times_predict[0], times_predict[-1]))



del data
del mesh
gc.collect()

HFM data has 3001 snapshots of dimension 14605 for h,u and v, spanning times [0.0, 6.0]

-------Prepare training and testing data---------
Using 313 training snapshots for time interval [2.5,4.996]
Using 1750 testing snapshots for time interval [2.5,5.998]


190

In [4]:
### ------ Compute the POD basis using the training snapshots------------------
trunc_lvl = 0.99
snap_norm, snap_mean, U, D, W = pod.compute_pod_multicomponent(snap_train)
nw, U_r = pod.compute_trunc_basis(D, U, eng_cap = trunc_lvl)

### ------ Compute the POD coefficients for training snapshots------------------
Z_train = pod.project_onto_basis(snap_train, U_r, snap_mean)


### ------ Compute the POD coefficients for the truth snapshots on the prediction interval------------------
Z_pred_true = pod.project_onto_basis(snap_pred_true, U_r, snap_mean)

npod_total = 0
for key in soln_names:
    npod_total+=nw[key]

p truncation level for 99.0% = 5, \sigma_6 = 17.58326064444933
v_x truncation level for 99.0% = 8, \sigma_9 = 10.971372231083336
v_y truncation level for 99.0% = 7, \sigma_8 = 13.390899418495513


In [18]:
### Setup NODE input data

reload(nd)
NODE = nd.NODEBase(device=device)
true_state_array, true_pred_state_array, init_state, state_len, dt_train, dt_predict = \
        NODE.prepare_input_data(Z_train, nw, times_train, stack_order, times_predict, Z_pred_true)

print("Training NODE using %d modes for %d time steps with %.3f <= t <= %.3f and dt = %.4f"%(state_len,
                             true_state_array.shape[0], times_train[0], 
                             times_train[-1], dt_train))
print("Predicting NODE solutions using %d modes for %d time steps with %.3f <= t <= %.3f and dt = %.4f"%(
                            state_len, true_pred_state_array.shape[0], times_predict[0], 
                            times_predict[-1], dt_predict))

Training NODE using 20 modes for 313 time steps with 2.500 <= t <= 4.996 and dt = 0.0080
Predicting NODE solutions using 20 modes for 1750 time steps with 2.500 <= t <= 5.998 and dt = 0.0020


In [19]:
reload(node)
### Preprocess training data (scale time and/or states, augment states if using ANODE)
### Set up learning rate scheduler and optimizer for training of the NODE model

true_state_tensor, times_tensor, init_state, learn_rate, optim = \
                NODE.preprocess_data(scale_states=scale_states, augmented=augmented, 
                        lr_decay=learning_rate_decay, init_lr=initial_learning_rate, opt=optimizer, 
                        scaling_method='centered', aug_dim=aug_dims, 
                        decay_steps=decay_steps, decay_rate=decay_rate, staircase=staircase_opt, )

In [35]:
### ---- Model Training ------
reload(node)
reload(nd)
train_loss_results, train_lr, saved_ep = \
         NODE.train_model(true_state_tensor, times_tensor, init_state, epochs, savedir,
                    solver=solver, purpose=purpose, adjoint=use_adjoint, minibatch=use_minibatch)



------------Begin training---------

Epoch 1: Loss = 0.878705, LR = 0.001000

Epoch 2: Loss = 0.506129, LR = 0.001000

******Saving model state. Epoch 2******

Epoch 3: Loss = 0.376773, LR = 0.001000

Epoch 4: Loss = 0.329251, LR = 0.001000

******Saving model state. Epoch 4******

Epoch 5: Loss = 0.305369, LR = 0.001000

Epoch 6: Loss = 0.288611, LR = 0.001000

******Saving model state. Epoch 6******

Epoch 7: Loss = 0.279439, LR = 0.001000

Epoch 8: Loss = 0.275501, LR = 0.001000

******Saving model state. Epoch 8******

Epoch 9: Loss = 0.273509, LR = 0.001000

Epoch 10: Loss = 0.271870, LR = 0.001000

******Saving model state. Epoch 10******

****Total training time = 15.964924335479736****

